In [1]:
import os 
from pathlib import Path
import sys
import subprocess
import time
from typing import Optional
from collections import defaultdict 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from IPython.display import display
import latextable
from texttable import Texttable
from arguments_dict import arguments_dict

executables = ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-xmm']
executable_paths_dict = dict([(exe, Path("executables") / exe) for exe in executables])
tests = dict([(t, Path('data-structures') / t / 'variants' / (os.listdir(Path('data-structures') / t / 'variants')[0])) for t in os.listdir('data-structures') if os.path.isdir(Path('data-structures') / t)])
subprocess_timeout = 30 #s
test_arguments = {
    "spinlock": "N=4",
    "ttaslock": "N=4",
    "barrier": "NUMREADERS=5", 
    "mutex": "N=2",
    "fcombiner-async": "N_THREADS=2",
    "dq": "",
    "chase-lev": "CONFIG_DEQ_STEALERS=2",
    "twalock": "N=4",
    "ticketlock": "N=6",
    "stc": "N=6",
    "stc-opt": "",
    "linuxrwlocks": "CONFIG_RWLOCK_READERS=2 CONFIG_RWLOCK_WRITERS=3",
    "ms-queue": "CONFIG_QUEUE_READERS=2 CONFIG_QUEUE_WRITERS=2",
    "treiber-stack": "CONFIG_STACK_READERS=2 CONFIG_STACK_WRITERS=3",
    "mpmc-queue": "CONFIG_MPMC_READERS=3 CONFIG_MPMC_WRITERS=2",
    "buf_ring": "",
    "mcs_spinlock": "N=4",
    "seqlock": "CONFIG_SEQ_READERS=2 CONFIG_SEQ_WRITERS=2",
    "seqlock-atomic": "CONFIG_SEQ_READERS=2 CONFIG_SEQ_WRITERS=2",
    "barrier-bnd": "NUMREADERS=4",
    "mpmc-queue-bnd": "",
    "seqlock-bnd": "CONFIG_LOCK_READERS=0 CONFIG_LOCK_WRITERS=3 CONFIG_LOCK_RDWR=0",
    # linuxrwlocks-bnd segfaults
    # treiber-stack-bnd llvm15 crashes
}

class RunExecutableResult:
    class Timeout:
        pass
    class Ok:
        def __init__(self, output) -> None:
            super().__init__()
            self.output = output
    class Error:
        def __init__(self, retcode, output) -> None:
            super().__init__()
            self.retcode = retcode
            self.output = output

def run_executable(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    try:
        result = subprocess.run([exe_path, *args, test_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE, timeout=subprocess_timeout)
        result_stdout = result.stdout.decode()
        result_stderr = result.stderr.decode()
        if result.returncode == 0:
            return RunExecutableResult.Ok(result_stdout + result_stderr)
        else:
            return RunExecutableResult.Error(result.returncode, result_stdout + result_stderr)
    except subprocess.TimeoutExpired:
        return RunExecutableResult.Timeout()

In [2]:
def get_execution_time(string: str) -> Optional[float]:
    try:
        pattern = "Total wall-clock time: "
        time = string.split(pattern)[1].split('s\n')[0]
        return float(time)
    except:
        return None
assert(get_execution_time(
"""
No errors were detected.
Number of complete executions explored: 3
Total wall-clock time: 0.05s
""") == 0.05)

def get_number_executions(string: str) -> Optional[int]:
    pattern = "Number of complete executions explored: "
    i = string.index(pattern) + len(pattern)
    number_str = ""
    while i < len(string) and string[i].isdigit():
        number_str += string[i]
        i += 1
    try:
        return int(number_str)
    except:
        return None

assert(get_number_executions("abc 123 Number of complete executions explored: 123456") == 123456)

def get_duplicate_executions(string: str) -> Optional[int]:
    try:
        pattern = "Number of duplicate executions explored: "
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_duplicate_executions("abc 123 Number of duplicate executions explored: 123456") == 123456)

def get_number_lb_races(string: str) -> Optional[int]:
    pattern = "Number of load buffering races explored: "
    try:
        i = string.index(pattern) + len(pattern)
        number_str = ""
        while i < len(string) and string[i].isdigit():
            number_str += string[i]
            i += 1
        return int(number_str)
    except:
        return None

assert(get_number_lb_races("abc 123 Number of load buffering races explored: 123456") == 123456)

class ResultItem:
    def __init__(self, execs, exe_time, dups, lb_races) -> None:
        self.execs = execs
        self.exe_time = exe_time
        self.dups = dups
        self.lb_races = lb_races

def run_and_get_results(exe_path: Path, args: list[str], test_path: Path) -> RunExecutableResult:
    res = run_executable(exe_path, args, test_path)
    if isinstance(res, RunExecutableResult.Ok):
        execs = get_number_executions(res.output)
        exe_time = get_execution_time(res.output)
        dups = get_duplicate_executions(res.output)
        lb_races = get_number_lb_races(res.output)
        if execs is None:
            # print(f"could not get number of executions in {res.output}")
            execs = 0
        if dups is None:
            dups = 0
        if exe_time is None:
            # print(f"could not get duplicate executions in {res.output}")
            exe_time = 0
        return RunExecutableResult.Ok(ResultItem(execs, exe_time, dups, lb_races))
    else:
        return res

In [3]:
timeout = '{\\fontspec{Symbola}\\symbol{"231B}}'
rows = [["Test Name", "Old GenMC Execs",  "Old GenMC Duplicates", "Old GenMC Time", "WMC Execs", "WMC Duplicates", "WMC Time", "GenMC Execs", "GenMC Duplicates", "GenMC Time", "XMM Execs", "XMM Duplicates", "XMM Time", "LB races"]]
assert(executables == ['genmc-old', 'genmc-wkmo', 'genmc', 'genmc-xmm'])
for test_name in test_arguments.keys():
    row = [f"{test_name}"]
    for exe in executables:
        if test_name in test_arguments:
            args = arguments_dict[exe] + ["--"] + [f"-D{a}" for a in test_arguments[test_name].split()]
        else:
            args = arguments_dict[exe]
        res = run_and_get_results(executable_paths_dict[exe], args, tests[test_name])
        if isinstance(res, RunExecutableResult.Ok):
            execs = res.output.execs
            exe_time = res.output.exe_time      
            dups = res.output.dups          
            row += [execs, dups, f"{exe_time:.2f}s"]
            if res.output.lb_races is not None:
                row += [res.output.lb_races]
        elif isinstance(res, RunExecutableResult.Timeout):
            row += [timeout] * 3
        elif isinstance(res, RunExecutableResult.Error):
            print(f"{exe} {test_name} error {res.retcode}: {res.output[:100]}")
            # row += [res.output[:60] if res.output != "" else f"error code {res.retcode}"] * 2
            row = None
            break
        else:
            assert(0)
    
    rows.append(row) if row is not None else None

# assert(len(rows) == 16)
print("rows has len", len(rows))

genmc-xmm twalock error 42: Error: Safety violation!
Event (4, 5) in graph:
XMM Execution Graph:
<-1, 0> main:
	(0, 1): MALLOC t
genmc-xmm stc error -11: 
genmc-xmm stc-opt error -11: 
genmc-xmm linuxrwlocks error -11: 
genmc-old ms-queue error 42: Error detected: Safety violation!
Event (3, 14) in graph:
<-1, 0> main:
	(0, 0): B
	(0, 1): Wna (que
genmc-old treiber-stack error 42: Error detected: Attempt to access non-allocated memory!
Event (0, 34) in graph:
<-1, 0> main:
	(0, 0
rows has len 17


In [4]:
data = rows[1:]
data.sort(key=lambda x: x[-1] if isinstance(x[-1], int) else -1, reverse=True)

for i in range(len(data)):
    data[i][0] = data[i][0].replace("_", "-")

data = [d for d in data if len(d) == len(rows[0])]

rows = [rows[0]] + data

In [5]:

time_rows = [[row[0]] + row[3:-1][0::3] + [row[-1]] for row in rows]

for x in time_rows:
    assert(len(x) == len(time_rows[0]))

table_1 = Texttable()
table_1.set_cols_align(["l"] + ["c" for i in range(len(time_rows[0]) - 1)])
table_1.set_cols_valign(["m" for i in range(len(time_rows[0]))])
table_1.add_rows(time_rows)

In [6]:
exec_rows = [[row[0]] + row[1:-1][0::3] + [row[-1]] for row in rows] 

for i in range(1, len(exec_rows)):
    execs = exec_rows[i][1]
    if any(p != execs for p in exec_rows[i][1:-1]):
        exec_rows[i][0] = f"\\textcolor{{red}}{{{exec_rows[i][0]}}}"

table_2 = Texttable()
table_2.set_cols_align(["l"] + ["c" for i in range(len(exec_rows[0]) - 1)])
table_2.set_cols_valign(["m" for i in range(len(exec_rows[0]))])
table_2.add_rows(exec_rows)

In [7]:
dups_rows = [[row[0]] + row[2:-1][0::3] + [row[-1]] for row in rows]

table_3 = Texttable()
table_3.set_cols_align(["l"] + ["c" for i in range(len(dups_rows[0]) - 1)])
table_3.set_cols_valign(["m" for i in range(len(dups_rows[0]))])
table_3.add_rows(dups_rows)

In [8]:
print(latextable.draw_latex(table_2, position='H', caption=f"Number of executions explored on data structure benchmarks run with the same version of GenMC that WMC is based on, WMC, GenMC, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:data-structures-benchmarks-execs"))
print(latextable.draw_latex(table_1, position='H', caption=f"Execution time on data structure benchmarks run with the same version of GenMC that WMC is based on, WMC, GenMC, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:data-structures-benchmarks-time"))
# print(latextable.draw_latex(table_3, position='H', caption=f"Duplicate executions on data structure benchmarks run with the same version of GenMC that WMC is based on, WMC, GenMC, and GenMC-XMM. Timeout was set to {subprocess_timeout}s", label="table:data-structures-benchmarks-dups"))

\begin{table}[H]
	\begin{center}
		\begin{tabular}{|l|c|c|c|c|c|}
			\hline
			Test Name & Old GenMC Execs & WMC Execs & GenMC Execs & XMM Execs & LB races \\
			\hline
			mpmc-queue-bnd & 15752 & 15752 & 15752 & 15752 & 24240 \\
			\hline
			ticketlock & 720 & 720 & 720 & 720 & 10800 \\
			\hline
			\textcolor{red}{chase-lev} & 3809 & 3975 & 3639 & 4070 & 7122 \\
			\hline
			buf-ring & 1218 & 1218 & 1218 & 1218 & 2172 \\
			\hline
			\textcolor{red}{dq} & 1802 & 1802 & 1924 & 1924 & 1478 \\
			\hline
			\textcolor{red}{fcombiner-async} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} & 24 & 24 & 32 \\
			\hline
			mutex & 12 & 12 & 12 & 12 & 30 \\
			\hline
			\textcolor{red}{spinlock} & 12096 & 12096 & 576 & 576 & 0 \\
			\hline
			\textcolor{red}{ttaslock} & 20760 & 20760 & 576 & 576 & 0 \\
			\hline
			barrier & 720 & 720 & 720 & 720 & 0 \\
			\hline
			\textcolor{red}{mcs-spinlock} & {\fontspec{Symbola}\symbol{"231B}} & {\fontspec{Symbola}\symbol{"231B}} 

In [9]:
print(table_1.draw())

+--------------+--------------+-------------+------------+----------+----------+
|  Test Name   |  Old GenMC   |  WMC Time   | GenMC Time | XMM Time | LB races |
|              |     Time     |             |            |          |          |
+==============+==============+=============+============+==========+==========+
| mpmc-queue-  |    2.60s     |    2.74s    |   3.10s    |  10.68s  |  24240   |
| bnd          |              |             |            |          |          |
+--------------+--------------+-------------+------------+----------+----------+
| ticketlock   |    3.10s     |    3.41s    |   0.29s    |  1.83s   |  10800   |
+--------------+--------------+-------------+------------+----------+----------+
| chase-lev    |    0.56s     |    0.57s    |   0.69s    |  9.77s   |   7122   |
+--------------+--------------+-------------+------------+----------+----------+
| buf-ring     |    1.12s     |    1.26s    |   1.39s    |  2.10s   |   2172   |
+--------------+------------

In [10]:
print(table_2.draw())

+-------------+-------------+-------------+-------------+-----------+----------+
|  Test Name  |  Old GenMC  |  WMC Execs  | GenMC Execs | XMM Execs | LB races |
|             |    Execs    |             |             |           |          |
+=============+=============+=============+=============+===========+==========+
| mpmc-queue- |    15752    |    15752    |    15752    |   15752   |  24240   |
| bnd         |             |             |             |           |          |
+-------------+-------------+-------------+-------------+-----------+----------+
| ticketlock  |     720     |     720     |     720     |    720    |  10800   |
+-------------+-------------+-------------+-------------+-----------+----------+
| \textcolor{ |             |             |             |           |          |
| red}{chase- |    3809     |    3975     |    3639     |   4070    |   7122   |
| lev}        |             |             |             |           |          |
+-------------+-------------

In [11]:
print(table_3.draw())

+-------------+-------------+-------------+------------+------------+----------+
|  Test Name  |  Old GenMC  |     WMC     |   GenMC    |    XMM     | LB races |
|             | Duplicates  | Duplicates  | Duplicates | Duplicates |          |
+=============+=============+=============+============+============+==========+
| mpmc-queue- |      0      |      0      |     0      |     0      |  24240   |
| bnd         |             |             |            |            |          |
+-------------+-------------+-------------+------------+------------+----------+
| ticketlock  |      0      |      0      |     0      |     0      |  10800   |
+-------------+-------------+-------------+------------+------------+----------+
| chase-lev   |      0      |      0      |     0      |   29047    |   7122   |
+-------------+-------------+-------------+------------+------------+----------+
| buf-ring    |      0      |      0      |     0      |     0      |   2172   |
+-------------+-------------